In [3]:
import cv2, os, time
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [20]:
class GetData:
    @staticmethod
    def get_images(folder_path):
        data_folder= os.path.join(os.curdir, folder_path)
        data_set = [os.path.join(data_folder,file) for file in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder,file))]
        images = [cv2.imread(x , cv2.IMREAD_GRAYSCALE) for x in data_set]
        return images

    @staticmethod
    def get_video_cuts(video_path, output_path):
        vidcap = cv2.VideoCapture(video_path)
        success,image = vidcap.read()
        count = 0
        success = True
        while success and count < 100:
            cv2.imwrite(output_path + "/frame%d.jpg" % count, image) 
            success,image = vidcap.read()
            count += 1
        return DataHandler.get_images(output_path)    

In [6]:
detector = cv2.AKAZE_create()
model = RandomForestClassifier()

images_good = GetData.get_images('train_true')
images_other = GetData.get_images('train_false')

X = []
Y = []

for i in images_good:
    kps, desc = detector.detectAndCompute(i, None)
    try:
        if len(desc) > 30:
            X.append(desc[:30])
            Y.append(1)
        else:
            print("Not pass")
    except Exception:
        print("Oops...Something goes wrong!")

for i in images_other:
    kps, desc = detector.detectAndCompute(i, None)
    try:
        if len(desc) > 30:
            X.append(desc[:30])
            Y.append(0)
        else:
            print("Not pass")
    except Exception:
        print("Oops...Something goes wrong!")



X = np.array(X)
X = X.reshape(X.shape[0], -1)
Y = np.array(Y)

model.fit(X, Y)


C:\Users\Danul\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
model.predict(X)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
images_good = GetData.get_images('test_true')
images_other = GetData.get_images('test_false')

X = []
X1= []


for i in images_good:
    _, desc = detector.detectAndCompute(i, None)
    try:
        if len(desc) > 30:
            X.append(desc[:30])
        else:
            print("Not pass")
    except Exception:
        print("Oops...Something goes wrong!")

for i in images_other:
    _, desc = detector.detectAndCompute(i, None)
    try:
        if len(desc) > 30:
            X1.append(desc[:30])
        else:
            print("Not pass")
    except Exception:
        print("Oops...Something goes wrong!")

X = np.array(X)
X = X.reshape(X.shape[0], -1)

X1 = np.array(X1)
X1 = X1.reshape(X1.shape[0], -1)



In [22]:
start = time.time()
x_predict_true = model.predict(X) # test_true
x_predict_false = model.predict(X1) # test_false
time_spend = time.time() - start # время

print(time_spend)
print("Помилок першого роду = ", int(np.sum(x_predict_false)))
print("Помилок другого роду = ", int(x_predict_true.size - np.sum(x_predict_true)))
print("Всего распознано = ", x_predict_true.size + x_predict_false.size, "из", 179)

0.00299835205078125
Помилок першого роду 36
Помилок другого роду 37
Всего распознано 179 из 179


In [13]:
x_predict_false

array([1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1])

In [14]:
x_predict_true

array([0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0])

In [21]:
GetData.get_video_cuts('video.mp4', 'video_cuts')

[array([[87, 87, 87, ..., 53, 55, 58],
        [87, 87, 87, ..., 54, 55, 56],
        [87, 87, 86, ..., 55, 54, 54],
        ...,
        [80, 78, 75, ..., 35, 33, 34],
        [79, 72, 73, ..., 37, 33, 29],
        [83, 73, 73, ..., 37, 31, 25]], dtype=uint8),
 array([[84, 82, 81, ..., 56, 53, 51],
        [83, 82, 81, ..., 56, 54, 51],
        [82, 82, 82, ..., 56, 54, 52],
        ...,
        [75, 79, 82, ..., 32, 30, 29],
        [83, 80, 74, ..., 31, 32, 32],
        [80, 80, 75, ..., 32, 34, 36]], dtype=uint8),
 array([[70, 70, 70, ..., 49, 49, 49],
        [70, 70, 70, ..., 49, 49, 49],
        [70, 70, 70, ..., 49, 49, 49],
        ...,
        [73, 73, 73, ..., 37, 40, 44],
        [65, 65, 66, ..., 40, 41, 40],
        [59, 60, 61, ..., 39, 37, 33]], dtype=uint8),
 array([[70, 70, 70, ..., 50, 51, 51],
        [70, 70, 70, ..., 50, 51, 51],
        [70, 70, 70, ..., 50, 51, 51],
        ...,
        [76, 77, 76, ..., 32, 32, 32],
        [74, 75, 75, ..., 29, 30, 31],
      

In [23]:
video_cuts = GetData.get_images('test_true')

X = []

for i in video_cuts:
    _, desc = detector.detectAndCompute(i, None)
    try:
        if len(desc) > 30:
            X.append(desc[:30])
        else:
            print("Not pass")
    except Exception:
        print("Oops...Something goes wrong!")


X = np.array(X)
X = X.reshape(X.shape[0], -1)

In [24]:
model.predict(X)

array([0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0])